In [298]:
from extract_data import extarct_data
from config.config import config

In [299]:
df = extarct_data('regression').drop(['title','equipement','id','city id','year','index'],axis=1)


In [300]:
#df.columns.to_list()
#df['ville'].value_counts()
display(df)

,price,nb rooms,nb baths,surface area,ville,jour,month,Articles De Toilettage,Ascenseur,Balcon,...,Fer À Repasser,Garage,Machine À Laver,Meublé,Parking,Serviettes,Sécurité,Terrasse,Télévision,WIFI
0,-2.211455,-2.293100,-0.894736,-0.597365,26,14,7,0,0,1,...,0,0,0,1,0,0,1,0,0,0
1,-2.230049,-0.483449,0.520454,0.246433,26,4,5,0,1,1,...,0,0,0,1,1,0,1,0,0,0
2,-2.431082,3.107139,0.520454,0.222611,14,1,8,0,1,1,...,0,0,0,0,1,0,1,1,0,0
3,-2.434868,-0.483449,0.520454,0.222611,14,15,6,1,1,1,...,1,0,1,1,1,1,1,1,1,1
4,0.008557,0.905145,0.520454,0.577952,18,15,6,0,0,1,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,-0.259262,-0.483449,0.520454,0.048865,37,18,2,0,1,1,...,0,0,0,1,1,0,1,0,0,0
1266,-0.245107,-0.483449,12.150805,-0.194819,4,14,10,0,1,1,...,0,0,0,0,1,0,1,1,0,0
1267,0.217656,-0.483449,0.520454,0.149687,22,20,10,0,1,1,...,0,0,0,1,1,0,1,0,0,0
1268,-0.346169,-2.293100,-0.894736,-0.281889,26,15,11,0,0,0,...,0,0,1,1,1,0,1,1,0,0


In [301]:
df[config.operation_array].corr()

,price,nb rooms,nb baths,surface area
price,1.000000,0.210319,0.243326,0.443951
nb rooms,0.210319,1.000000,0.330073,0.467571
nb baths,0.243326,0.330073,1.000000,0.345295
surface area,0.443951,0.467571,0.345295,1.000000


In [302]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

x,y = df[[
 'nb rooms',
 'nb baths',
 'surface area',
 'ville',
 'Ascenseur'
 
]],df['price']

#display(x)
#display(y)


In [303]:
from sklearn.model_selection import train_test_split

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(x)

model = LinearRegression(n_jobs=-1)
x_train,x_test,y_train,y_test = train_test_split(X_poly,y,test_size=0.2, random_state=42)

model.fit(x_train,y_train)


LinearRegression(n_jobs=-1)

In [304]:
model.score(x_test,y_test)

0.27776088616061634

In [305]:
from sklearn.metrics import mean_squared_error, r2_score


y_pred=model.predict(x_test)

mse = mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)

print(mse)
print(r2)

0.5931210370562907
0.27776088616061634


In [306]:
from sklearn.ensemble import RandomForestRegressor


x,y = df[[
 'nb rooms',
 'nb baths',
 'surface area',
 'ville',
 'Ascenseur'
 
]],df['price']

X_train, X_test, y_train, y_test = train_test_split(x ,y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  
rf_model.fit(X_train, y_train)





RandomForestRegressor(random_state=42)

In [307]:
y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)  
r2 = r2_score(y_test, y_pred)

In [308]:
print(mse)

0.7486285884546706


In [309]:
print(r2)

0.08840048735446315


In [310]:
from sklearn.model_selection import GridSearchCV

grid_param = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 4],
    'max_features': ['auto', 'sqrt'],
    'bootstrap': [True, False]
}

model = RandomForestRegressor(n_jobs=-1)
grid_model = GridSearchCV(model,param_grid=grid_param,cv=5)
grid_model.fit(x_train,y_train)


/home/ali/Desktop/machine learning /venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
240 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ali/Desktop/machine learning /venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ali/Desktop/machine learning /venv/lib/python3.12/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/ali/Desktop/machine learning /venv/lib/python3.12/site-packages/sklearn/base.py", line 6

GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1),
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 4],
                         'min_samples_split': [2, 10],
                         'n_estimators': [100, 200]})

In [311]:
grid_model.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 100}

In [312]:
model = grid_model.best_estimator_

In [313]:
model.score(x_test,y_test)

0.2964897372222707

In [315]:
y_pred = model.predict(x_test)


In [316]:
mse = mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)
print(mse)
print(r2)

0.5777404306176429
0.2964897372222707
